In [7]:
import pandas as pd
import numpy as np
import os

prepare_RSgroups = False #надо ли объединить группы в один файл
prepare_BOgroups = False

# dataset_name = 'prepared_facebook_data'
# folder_name = 'facebook_comm_vol'
# dataset_name ='house_prices'
# folder_name = 'boston_house_prices'
# dataset_name = 'physics'
# folder_name = 'physics'
# dataset_name = 'energy_consumption'
# folder_name = 'energy_consumption'
dataset_name = 'quote'
folder_name = 'quote'

DIR_WITH_FILES = '../other_data_experiments/{}/first experiment/'.format(folder_name)
# DIR_WITH_BO_FILES =  '../other_data_experiments/{}/third experiment/groups/'.format(folder_name)

DIR_TO_SAVE_RES = DIR_WITH_FILES
# DIR_TO_SAVE_RES = DIR_WITH_BO_FILES

In [4]:
if prepare_RSgroups:
    pathname = "res_rand_group"
elif prepare_BOgroups:
    pathname = "res_BO_group"
else:
    pathname = None
    
if pathname:
    for i in range(1,11):
        try:
            part_1 = pd.read_csv(DIR_TO_SAVE_RES+\
                                 "{}1_{}_{}.csv".format(pathname, dataset_name, i))
            part_2 = pd.read_csv(DIR_TO_SAVE_RES+\
                                 "{}2_{}_{}.csv".format(pathname, dataset_name, i))
            part_3 = pd.read_csv(DIR_TO_SAVE_RES+\
                                 "{}3_{}_{}.csv".format(pathname, dataset_name, i))
            full = pd.concat([part_1, part_2, part_3], sort = False)
            full['run_number']=i
            print(full.shape)
            full.to_csv(DIR_TO_SAVE_RES+"{}s_{}_{}.csv".format(pathname, dataset_name, i), index=False)
        except:
            print(f"{i} is NONE")

(729, 19)
(729, 19)
(729, 19)
(729, 19)
(729, 19)
(729, 19)
(729, 19)
(729, 19)
(729, 19)
(729, 19)


In [ ]:
RS_time = np.array([])
print(np.mean(RS_time))
print(np.std(RS_time))

In [ ]:
group_1_time = np.array([])
group_2_time = np.array([])
group_3_time = np.array([])

sum_arr=np.sum([group_1_time, group_2_time, group_3_time], axis = 0)


for arr in [group_1_time, group_2_time, group_3_time, sum_arr]:
    print(np.mean(arr))
    print(np.std(arr))
    print("=====")

In [2]:
def group_best_params_with_stop(df, stop_iter_num, left_iter_num, right_iter_num):
    best_params = {}
    stop_iter_nums = {}
    best_score = {}

    tmp_df = df.loc[(df['iter_num'] >= left_iter_num) & (df['iter_num'] <= right_iter_num)].copy() #take group
    for i in tmp_df.run_number.unique(): 
      tmp_df_i = tmp_df.loc[tmp_df.run_number==i] #take run
      #cut iterations:
      tmp_df_i_copy = tmp_df_i.copy()
      tmp_df_i_copy.loc[:, "mean_test_score"] = tmp_df_i_copy.loc[
          :, "mean_test_score"].cummax()      

      try:
          curr_max=tmp_df_i_copy.groupby("mean_test_score").count().reset_index().sort_values(by = "mean_test_score")
          max_score = curr_max.loc[curr_max["mean_test_score"]>=stop_iter_num].index[0] #get first interval with iter_amount more than stop_iter_num
      except:
          max_score = tmp_df_i_copy.mean_test_score.max()

      tmp_stop = tmp_df_i.loc[tmp_df_i["mean_test_score"]==max_score].sort_values(by="iter_num").iloc[0]#get first element with best_score
    
      best_params[i] = tmp_stop['params']
      #best_score[i] = tmp_stop['mean_test_score']

      last_iter_num = tmp_stop['iter_num'] + stop_iter_num - 1
      if last_iter_num > right_iter_num:
          last_iter_num = right_iter_num

      stop_iter_nums[i] = last_iter_num
    return best_params, stop_iter_nums#, best_score       
   

In [6]:
group_df = pd.DataFrame([])

pathname =  "res_rand_group" #"res_BO_group" 
for i in range(1,11):
    try:
        df = pd.read_csv(DIR_TO_SAVE_RES+"{}s_{}_{}.csv".format(pathname, dataset_name, i))    
        df['iter_num'] = range(1, df.shape[0]+1)
        group_df = pd.concat([group_df, df], sort = False)
        print(f"{i} is OK")
    except:
        print(f"{i} is NONE")

1 is OK
2 is OK
3 is OK
4 is OK
5 is OK
6 is OK
7 is OK
8 is OK
9 is OK
10 is OK


In [5]:
stop_iter_num = 10
# pathname =  "res_rand_group" 
best_params_first_group, first_stop_iter_nums = group_best_params_with_stop(
    group_df, stop_iter_num = stop_iter_num,
    left_iter_num=1, right_iter_num=243)

In [8]:
best_params_first_group

{1: "OrderedDict([('max_depth', 5), ('min_child_weight', 5)])",
 2: "OrderedDict([('max_depth', 5), ('min_child_weight', 8)])",
 3: "OrderedDict([('max_depth', 5), ('min_child_weight', 8)])",
 4: "OrderedDict([('max_depth', 5), ('min_child_weight', 8)])",
 5: "OrderedDict([('max_depth', 4), ('min_child_weight', 5)])",
 6: "OrderedDict([('max_depth', 5), ('min_child_weight', 8)])",
 7: "OrderedDict([('max_depth', 5), ('min_child_weight', 8)])",
 8: "OrderedDict([('max_depth', 5), ('min_child_weight', 8)])",
 9: "OrderedDict([('max_depth', 5), ('min_child_weight', 8)])",
 10: "OrderedDict([('max_depth', 5), ('min_child_weight', 8)])"}

In [9]:
first_stop_iter_nums

{1: 75, 2: 212, 3: 136, 4: 144, 5: 158, 6: 90, 7: 181, 8: 91, 9: 220, 10: 159}

In [8]:
#stop_iter_num = 25

best_params_second_group={}
second_stop_iter_nums={}

full_datasets = [2]
for i in full_datasets:
    group_df = pd.read_csv(DIR_TO_SAVE_RES+"{}s_{}_{}.csv".format(pathname, dataset_name, i))
    group_df['iter_num'] = range(1, group_df.shape[0]+1)
    first_group_stop_iters = 243
    left_iter_num = first_group_stop_iters+1
    right_iter_num =left_iter_num + int(round((729 - first_group_stop_iters)/2))

    best_params_second_group_tmp, second_stop_iter_nums_tmp = group_best_params_with_stop(
        group_df, stop_iter_num = stop_iter_num,
        left_iter_num=left_iter_num, right_iter_num=right_iter_num)
    best_params_second_group.update(best_params_second_group_tmp)
    second_stop_iter_nums.update(second_stop_iter_nums_tmp)

In [9]:
print(best_params_second_group)

{2: "{'alpha': 0.41456378487232903, 'lambda': 0.5432846295868189}"}


In [10]:
print(second_stop_iter_nums)

{2: 484}


In [ ]:
# код склейки для 3 группы со второй, если первая группа была полной 

DIR_TO_SAVE_RES_SEC= '../other_data_experiments/{}/second experiment/'.format(folder_name)


In [ ]:
for params_set_num in full_datasets:
    res_rand_2 = pd.read_csv(
        DIR_TO_SAVE_RES+'res_rand_group1_{}_{}.csv'.format(dataset_name, params_set_num))
    
    res_rand_3 = pd.read_csv(
        DIR_TO_SAVE_RES_SEC+'GR_S_3_group_with_{}_{}_{}.csv'.format(stop_iter_num, params_set_num, dataset_name))
    
    best_params_third_group, third_stop_iter_nums = group_best_params_with_stop(res_rand_3, 
                                                          stop_iter_num = stop_iter_num, 
                                                          left_iter_num=second_stop_iter_nums[params_set_num]+1, 
                                                          right_iter_num=729)
    
    res_rand_3=res_rand_3.loc[(res_rand_3['run_number']==params_set_num) & \
                            (res_rand_3['iter_num']<=int(third_stop_iter_nums[params_set_num]))]

    
    res=pd.concat([res_rand_2,res_rand_3], sort=False)

    sec_idxs = res.loc[res['experiment_name']=='random search for 2 group','iter_num'].values
    third_idxs = res.loc[res['experiment_name']=='random search for 3 group','iter_num'].values
    intersection = np.intersect1d(third_idxs, sec_idxs)

    res = res.loc[~((res['experiment_name']=='random search for 2 group') & (res['iter_num'].isin(intersection)))]

    res.to_csv(DIR_TO_SAVE_RES_SEC+'GR_S_2_and_3_groups_with_{}_{}_{}.csv'.format(stop_iter_num, params_set_num, dataset_name),
               index=False)

In [ ]:
group_df = pd.read_csv("./data/tmp/GR_S_2_group_with_25_8_house_prices.csv")
# {'max_depth': 3, 'min_child_weight': 3}

stop_iter_num = 100

first_group_stop_iters = 109
left_iter_num = first_group_stop_iters+1
right_iter_num =left_iter_num + int(round((729 - first_group_stop_iters)/2))

best_params_second_group, second_stop_iter_nums = group_best_params_with_stop(
    group_df, stop_iter_num = stop_iter_num,
    left_iter_num=left_iter_num, right_iter_num=right_iter_num)

In [ ]:
best_params_second_group

In [ ]:
second_stop_iter_nums